In [65]:
import json
import redis
from rq import Queue

redis_client = redis.Redis(
    host="redis",
    port=6379,
)

q = Queue(connection=redis_client)

job = q.enqueue("abc", ttl=10)
print(vars(job))

{'connection': Redis<ConnectionPool<Connection<host=redis,port=6379,db=0>>>, '_id': '4218b686-ea77-4c5b-9777-02338548fab6', 'created_at': datetime.datetime(2022, 3, 20, 9, 11, 42, 885505), '_data': b'\x80\x05\x95\x0e\x00\x00\x00\x00\x00\x00\x00(\x8c\x03abc\x94N)}\x94t\x94.', '_func_name': 'abc', '_instance': None, '_args': (), '_kwargs': {}, '_success_callback_name': None, '_success_callback': <object object at 0x7fcbf0742740>, '_failure_callback_name': None, '_failure_callback': <object object at 0x7fcbf0742740>, 'description': 'abc()', 'origin': 'default', 'enqueued_at': datetime.datetime(2022, 3, 20, 9, 11, 42, 885828), 'started_at': None, 'ended_at': None, '_result': None, 'exc_info': None, 'timeout': 180, 'result_ttl': None, 'failure_ttl': None, 'ttl': 10, 'worker_name': None, '_status': <JobStatus.QUEUED: 'queued'>, '_dependency_ids': [], 'meta': {}, 'serializer': <class 'rq.serializers.DefaultSerializer'>, 'retries_left': None, 'retry_intervals': None, 'redis_server_version': (6

In [ ]:
import json
import redis

redis_client = redis.Redis(
    host="redis",
    port=6379,
)

redis_client

In [93]:
import io
from PIL import Image


img_path = "../../data/bobby.jpg"

def image_to_byte_array(image: Image, image_format: str):
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format=image_format)
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

with Image.open(img_path) as img:
    resized_img = img.resize((224, 224))
    img_bytes = image_to_byte_array(resized_img, img.format)
    print(len(img_bytes))
redis_client.set(img_bytes, "predict result")

6431


True

In [104]:
for i in range(10000):
    key = str(img_bytes) + str(i)
    redis_client.set(key, f"test_{i}")

test_key = str(img_bytes) + str(3000)

In [105]:
%%timeit
redis_client.get(test_key)

288 µs ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [106]:
redis_client.get(test_key)

b'test_3000'

In [80]:
i = io.BytesIO()
dir(i)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 'close',
 'closed',
 'detach',
 'fileno',
 'flush',
 'getbuffer',
 'getvalue',
 'isatty',
 'read',
 'read1',
 'readable',
 'readinto',
 'readinto1',
 'readline',
 'readlines',
 'seek',
 'seekable',
 'tell',
 'truncate',
 'writable',
 'write',
 'writelines']

In [54]:
redis_client.get("abcde")

In [34]:
redis_client.set(1, 'test1')

True

In [36]:
value = redis_client.get(1)
value

b'test1'

In [6]:
from pydantic import BaseModel, Field, validator
from typing import Optional

In [30]:
class Hoge(BaseModel):
    hoge: int

    @validator("hoge", pre=True)                # hogeのバリデーションの登録 
    def validate_hoge(cls, value):    # 関数名はなんでもいい。第1引数はcls固定で使用しない。第2引数はvalueでhogeに設定した値
        if isinstance(value, float):             # Noneであれば例外を投げる
            raise TypeError("none is not an allowed value") 
        return value                  # Noneでない場合はそのままvalueを返す

In [31]:
Hoge(hoge = 2.5)

ValidationError: 1 validation error for Hoge
hoge
  none is not an allowed value (type=type_error)

In [9]:
Hoge.hoge

'test'